In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
clinical_df = pd.read_excel('/media/tx-deepocean/Data/ChongFu1SOC/推想组学数据--总.xls', sheet_name='上皮来源')
# 去除患者姓名(pid标记即可）
clinical_df = clinical_df.iloc[:228, :].drop(['姓名', 'P号'], axis=1)
# clinical_df = clinical_df.iloc[:228, :].drop(['姓名', 'P号'], axis=1)
# 将中文改成英文
clinical_df = clinical_df.rename(columns={'通用编号':'pid', 'P号':'pid1', '减瘤情况':'RD', '年龄':'age', '单/双侧':'unilateral_or_bilateral',\
                    '肿块特征':'mass_feature','膈膜结节':'diaphram_nodule','直肠、乙状结肠关系分级（T1双回波）':'relationship_on_T1_dual_echo_images',\
                    '腹膜结节（或肿块）':'peritoneum_mesentery_nodules','腹腔内远处实质脏器实质转移':'parenchymal_organs', 'ASA分级':'ASA',\
                    '腹水量':'ascites_amount','CA125':'CA125', 'HE4':'HE4', 'LDH':'LDH', 'NLR':'NLR'})

clinical_df.insert(1, 'label', clinical_df['RD'].apply(lambda s: 0 if s == 'R0' else 1 if s=='R1' else 1))
clinical_df = clinical_df.drop(['RD'], axis=1)

def features_preprocess(df):
    # 去除重复项
    df = df.drop_duplicates()
    # 丢掉缺失值比例超过50%的列
    df = drop_col(df)
    # 用众数填充缺失值
    most_frequency = pd.Series(df.value_counts().index[0], index=df.columns)
    # 用中位数填充缺失值
    # most_frequency = df.median()
    df = df.fillna(most_frequency)
    return df

# 缺失值大于cut,则丢弃此列
def drop_col(df, cutoff=0.5):
    n = len(df)
    for col_name in df.columns:
        cnt = df[col_name].isnull().sum()
        if (float(cnt) / n) > cutoff:
            df.drop([col_name], axis=1, inplace=True)
    return df

clinical_df = features_preprocess(clinical_df)

int_col = []
for col in clinical_df.columns:
    num = len(clinical_df[col].value_counts())
    if num <= 4:
         int_col.append(col)
clinical_df[int_col] = clinical_df[int_col].astype('int32')
clinical_df['pid'] = clinical_df['pid'].astype('int')
clinical_df.at[150,'HE4'] = 1500
clinical_df.loc[clinical_df.pid==45, 'label']=0
clinical_df.to_csv('./dataset_info/clinical_data.csv', index=0)                 

In [4]:
print(clinical_df.shape)
clinical_df.head()

(228, 15)


,pid,label,age,unilateral_or_bilateral,mass_feature,diaphram_nodule,relationship_on_T1_dual_echo_images,peritoneum_mesentery_nodules,parenchymal_organs,CA125,HE4,LDH,NLR,ASA,ascites_amount
0,1,0,58.0,1,3,0,0,0,0,22.9,67,176.0,1.37,2,1
1,2,1,50.0,2,2,0,3,1,0,359.3,251,220.0,5.08,2,1
2,3,1,64.0,1,3,0,2,0,0,3440.9,293,234.0,2.97,2,1
3,4,0,61.0,2,3,0,3,0,0,778.1,609,534.0,11.34,2,1
4,5,1,49.0,2,2,0,0,1,0,2837.4,112,323.0,8.48,2,2


In [5]:
def OneHot(df,columns):
    df_new=df.copy()
    print(columns)
    for column in columns:
        value_sets=df_new[column].unique()
        for value_unique in value_sets:
            col_name_new=column + '_' + str(value_unique)
            df_new[col_name_new]=(df_new[column]==value_unique)
            df_new[col_name_new]=df_new[col_name_new].astype('int')
    return df_new

clinical_df1 = OneHot(clinical_df, int_col)
clinical_df1.to_csv('./dataset_info/clinical_onehot_data.csv',index=0)

['label', 'unilateral_or_bilateral', 'mass_feature', 'diaphram_nodule', 'relationship_on_T1_dual_echo_images', 'peritoneum_mesentery_nodules', 'parenchymal_organs', 'ASA', 'ascites_amount']


In [6]:
clinical_df1.head()

,pid,label,age,unilateral_or_bilateral,mass_feature,diaphram_nodule,relationship_on_T1_dual_echo_images,peritoneum_mesentery_nodules,parenchymal_organs,CA125,...,relationship_on_T1_dual_echo_images_2,relationship_on_T1_dual_echo_images_1,peritoneum_mesentery_nodules_0,peritoneum_mesentery_nodules_1,parenchymal_organs_0,parenchymal_organs_1,ASA_2,ASA_3,ascites_amount_1,ascites_amount_2
0,1,0,58.0,1,3,0,0,0,0,22.9,...,0,0,1,0,1,0,1,0,1,0
1,2,1,50.0,2,2,0,3,1,0,359.3,...,0,0,0,1,1,0,1,0,1,0
2,3,1,64.0,1,3,0,2,0,0,3440.9,...,1,0,1,0,1,0,1,0,1,0
3,4,0,61.0,2,3,0,3,0,0,778.1,...,0,0,1,0,1,0,1,0,1,0
4,5,1,49.0,2,2,0,0,1,0,2837.4,...,0,0,0,1,1,0,1,0,0,1


In [7]:
clinical_df2 = clinical_df[(clinical_df['peritoneum_mesentery_nodules']==1) &(clinical_df['mass_feature'] == 1)]

len(clinical_df2)

20

## External dataset

In [8]:
external_df = pd.read_excel('/media/tx-deepocean/Data/2022/chongfu1/Dataset/20220331/external_test/卵巢癌外部验证数据.xls', sheet_name='Sheet1')
# 去除患者姓名(pid标记即可）
external_df = external_df.drop(['姓名', '检查号'], axis=1)
# external_df = external_df.iloc[:228, :].drop(['姓名', 'P号'], axis=1)
# 将中文改成英文
external_df = external_df.rename(columns={'编号':'pid', '检查号':'pid1', '减瘤情况':'RD', '年龄':'age', '单/双侧':'unilateral_or_bilateral',\
                    '肿块特征':'mass_feature','膈膜结节':'diaphram_nodule','肿块与直肠、乙状结肠关系':'relationship_on_T1_dual_echo_images',\
                    '腹膜结节或肿块':'peritoneum_mesentery_nodules','腹腔内远处实质脏器转移':'parenchymal_organs', 'ASA分级':'ASA',\
                    '腹水量':'ascites_amount','CA125':'CA125', 'HE4(pmol/L)':'HE4', 'LDH(U/L)':'LDH', 'NLR':'NLR'})

external_df.insert(1, 'label', external_df['RD'].apply(lambda s: 0 if s == 'R0' else 1 if s=='R1' else 1))
external_df = external_df.drop(['RD'], axis=1)

def features_preprocess(df):
    # 去除重复项
    df = df.drop_duplicates()
    # 丢掉缺失值比例超过50%的列
    df = drop_col(df)
    # 用众数填充缺失值
    most_frequency = pd.Series(df.value_counts().index[0], index=df.columns)
    # 用中位数填充缺失值
    # most_frequency = df.median()
    df = df.fillna(most_frequency)
    return df

# 缺失值大于cut,则丢弃此列
def drop_col(df, cutoff=0.5):
    n = len(df)
    for col_name in df.columns:
        cnt = df[col_name].isnull().sum()
        if (float(cnt) / n) > cutoff:
            df.drop([col_name], axis=1, inplace=True)
    return df

external_df = features_preprocess(external_df)

int_col = []
for col in external_df.columns:
    num = len(external_df[col].value_counts())
    if num <= 4:
         int_col.append(col)

external_df['pid'] = external_df['pid'].apply(lambda x: re.findall("\d+", x)[0]).astype('int32')
external_df[int_col] = external_df[int_col].astype('int32')
external_df['pid'] = external_df['pid'].astype('int')
external_df = external_df.iloc[:45,:]
external_df.loc[39,'label']=0
external_df.to_csv('./dataset_info/external_clinical.csv', index=0)                 

In [9]:
print(external_df.shape)
external_df.tail(7)

(45, 15)


,pid,label,age,unilateral_or_bilateral,mass_feature,diaphram_nodule,relationship_on_T1_dual_echo_images,peritoneum_mesentery_nodules,parenchymal_organs,CA125,HE4,LDH,NLR,ASA,ascites_amount
38,39,0,67.0,1,2,0,2,0,0,1384.22,179.3,242.7,2.04,2,2
39,40,0,51.0,2,2,0,2,1,0,110.13,129.6,132.2,4.19,2,1
40,41,0,52.0,2,2,0,0,0,0,133.48,1227,1222.2,2.40,2,1
41,42,0,47.0,2,2,1,2,1,0,500.3,74.06,395.4,5.51,2,2
42,43,0,46.0,2,3,1,1,1,0,3923.2,751.34,276.9,3.75,2,2
43,44,1,57.0,2,2,0,1,1,0,1372.1,1000,849.5,3.75,2,2
44,45,1,57.0,1,2,1,3,1,0,3763.4,749.36,215.7,4.13,2,2
